In [1]:
import pandas as pd
import numpy as np
import re
import enchant
import string
from collections import defaultdict

In [3]:
import os 
# os.environ["JAVA_HOME"] = "C:\\Program Files\\Microsoft\\jdk-11.0.12.7-hotspot\\bin"

In [2]:
english_dict = enchant.Dict('en_IN')

In [5]:
# if not pt.started():
#     pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [3]:
corpus_path = 'E:/Pycharm/IR/Experiment1/Mails'
no_documents = 100

In [4]:
words = set()
ordered_words = [[] for i in range(no_documents)]

In [5]:
icon_pattern = (r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-'
                r'\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF'
                r'\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-'
                r'\U0001FA6F\U0001FA70-\U0001FAFF\U0001FB00-\U0001FBFF'
                r'\U0001FC00-\U0001FCFF\U0001FD00-\U0001FDFF\U0001FE00-'
                r'\U0001FEFF\U0001FF00-\U0001FFFF]+')
translation_table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

In [ ]:
# pattern = r'[&#\n*/.,!?(){}<>\[\]"\'=;,@\d]'
# symbols_to_remove = r'[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'

In [6]:
for i in range(no_documents):
    with open(f'{corpus_path}/email_{i}.txt', 'r' ,encoding='utf-8') as file:
        text = file.read()
        text = text.translate(translation_table) # symbol removal
        text = re.sub(icon_pattern, '', text).split(' ') # icon removal
        for word in text:
            word = str(word).lower()
            word = re.sub(r'[\d]', '', word) # digit removal
            if word and english_dict.check(word): # valid english word checking
                if len(word) != 1:
                    ordered_words[i].append(word)
                    words.add(word)      

In [7]:
number_of_unique_words = len(list(words))
number_of_unique_words

2789

In [8]:
sorted_words = sorted(words)
bigram = pd.DataFrame(0, 
                      columns=list(sorted_words), 
                      index=list(sorted_words))

In [9]:
for docno in range(no_documents):
    for i in range(len(ordered_words[docno]) - 1):
        bigram.loc[ordered_words[docno][i + 1], ordered_words[docno][i]] += 1

In [10]:
word_sequences = defaultdict(int)
for docno in range(no_documents):
    for i in range(len(ordered_words[docno]) - 2):
        seq = tuple(ordered_words[docno][i: i + 3])
        word_sequences[seq] += 1

In [11]:
word_sequences

defaultdict(int,
            {('subject', 'event', 'wrap'): 2,
             ('event', 'wrap', 'up'): 4,
             ('wrap', 'up', 'institute'): 2,
             ('up', 'institute', 'of'): 2,
             ('institute', 'of', 'information'): 16,
             ('of', 'information', 'technology'): 16,
             ('information', 'technology', 'international'): 9,
             ('technology', 'international', 'campus'): 9,
             ('international', 'campus', 'introduction'): 2,
             ('campus', 'introduction', 'to'): 2,
             ('introduction', 'to', 'google'): 2,
             ('to', 'google', 'student'): 2,
             ('google', 'student', 'clubs'): 2,
             ('student', 'clubs', 'congratulations'): 1,
             ('clubs', 'congratulations', 'your'): 1,
             ('congratulations', 'your', 'latest'): 2,
             ('your', 'latest', 'institute'): 2,
             ('latest', 'institute', 'of'): 2,
             ('international', 'campus', 'event'): 2,
        

In [12]:
df = pd.DataFrame(list(word_sequences.items()), columns=['Word Sequence', 'Frequency'])
df[['Word1', 'Word2', 'Word3']] = pd.DataFrame(df['Word Sequence'].to_list(), index=df.index)
df.drop(columns=['Word Sequence'], inplace=True)
multi_index = pd.MultiIndex.from_frame(df[['Word1', 'Word2']])
trigram = df.set_index(multi_index).pivot(columns='Word3', values='Frequency').fillna(0)

In [13]:
def get_next_word_bigram(word: str):
    next_possible_words = bigram[bigram[word] > 0].index.tolist()
    if not next_possible_words: return None
    freqs = bigram.loc[next_possible_words, word].tolist() 
    weights = np.array(freqs) / sum(freqs)
    return np.random.choice(next_possible_words, p=weights)

In [14]:
def random_text_generator(text_length: int) -> str:
    random_num = np.random.randint(0, number_of_unique_words)
    random_text = [list(words)[random_num]]
    curr_word = random_text[0]  
    text_length -= 1 
    while text_length:
        curr_word = get_next_word_bigram(curr_word)
        if curr_word is None: break
        random_text.append(curr_word)
        text_length -= 1
    
    return ' '.join(random_text)  

In [15]:
def random_text_generator_trigram(text_length: int) -> str:
    # get the first word randomly from the list of words
    # for the second use the bigram
    # from then onwards use trigram for the third word
    random_num = np.random.randint(0, number_of_unique_words)
    random_text = [list(words)[random_num]]
    curr_word1 = random_text[0]
    if text_length == 1: return curr_word1
    curr_word2 = get_next_word_bigram(curr_word1)
    random_text.append(curr_word2)
    text_length -= 2
    while text_length:
        if (curr_word1, curr_word2) not in trigram.index:
            return ' '.join(random_text)
        df = trigram.loc[(curr_word1, curr_word2),:]
        next_possible_words = df[df > 0].dropna().index.tolist()
        if not next_possible_words: break
        freqs = df[df>0].dropna().values.tolist()
        weights = np.array(freqs) / sum(freqs)
        next_word = np.random.choice(next_possible_words, p=weights)
        random_text.append(next_word)
        curr_word1 = curr_word2
        curr_word2 = next_word
        text_length -= 1
        
    return ' '.join(random_text)
        

In [16]:
random_text_generator(10)

'compiled java has got meeting invite with more com email'

In [19]:
random_text_generator_trigram(10)

'flags fly high we celebrate pride month you are not'